In [13]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS


In [14]:
# pinecone:pcsk_5CLKXJ_2rLtN8LDmNfP9eZWynKKsiXEEXSmqcavP4xehcAnXXYw1GttocvtweX4pvoo1w7
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="pcsk_5CLKXJ_2rLtN8LDmNfP9eZWynKKsiXEEXSmqcavP4xehcAnXXYw1GttocvtweX4pvoo1w7")

g:\Code\Python\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


1. 文档分割
将大文档切分成小的、独立的块，以便更好地存储和检索。

方法：
固定长度分割：将文档按字符数、单词数或句子数切分。

2. 向量化并存入向量数据库
将分割后的文档块进行向量化，并存储到向量数据库中。



3. 查询时增强检索
利用输入查询，检索相关的文档块并作为上下文添加到 Prompt。

4. 生成回答
将上下文和查询组合，输入到语言模型中生成回答。

In [15]:
# Step 1: 文档加载
# 文件路径
txt_file_path = r'C:\Users\ryan0\OneDrive\Obsidian_Lib\11_GWU\13_24FA\CSCI_6365_A_ML\20241113_AML_Final_Proj\data\attraction_resource\DC_attraction.txt'

# 加载数据
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
data = loader.load()

# Step 2: 文本分割
# 分割器：设置分割块的大小和重叠部分的大小
text_splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=200)
split_data = text_splitter.split_documents(data)

# 打印分割后的前 3 个数据块
print("分割后的文档块:")
for i, chunk in enumerate(split_data[:3]):
    print(f"Chunk {i + 1}: {chunk}")
    print("-" * 80)

分割后的文档块:
Chunk 1: page_content='https://travel.usnews.com/Washington_DC/Things_To_Do/


U.S. News and World Report Logo
Travel
Home
Vacations
Travel Guides
Tours and Attractions
Hotels
Cruises
Rewards
Advice

Sign In

Washington, D.C.USATravel Guides
Washington, D.C. Travel Guide
USA  
#1 in Best Historical Cities to Visit in the USA
 Credit

Overview
Things to do
Hotels
Dining
When to visit
Getting around
Map & Neighborhoods
Photos
31 Best Things To Do in Washington, D.C.

By Erin Evans
|
Reviewed by Ann Henson
|
Last updated on Sep. 6, 2024
Many of Washington, D.C.'s main attractions relate to its principal enterprise: politics. These include the White House and the U.S. Capitol, of course, as well as monuments and historic sites dedicated to notable historical figures. The capital is also a destination for museumgoers. The Smithsonian Institute operates more than 20 museums (most free to visit!), including the National Museum of African American History and Culture, the National Air

In [16]:
# Step 3: 初始化向量化模型
# 使用 OpenAIEmbeddings，确保使用正确的导入路径和模型名称
embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002"  # 当前推荐的嵌入模型
)

# Step 4: 构建向量存储
# 将分割后的文档块存入 FAISS 向量数据库
vectorstore = FAISS.from_documents(split_data, embedding=embeddings)

# 验证向量存储中已加载的文档数量
loaded_doc_count = len(vectorstore.docstore._dict)  # `_dict` 存储文档的内部结构
print(f"向量存储已加载 {loaded_doc_count} 个文档块")

向量存储已加载 29 个文档块


In [17]:
index_name = "quickstart"

pc.create_index(
    name=index_name,
    dimension=2, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)